# Enter an Activity Id and See a Summary of that Activity

In [ ]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import ConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, ActivityLaps
from idbutils import Location

activity_id = input('Enter the id of the activity you would like to display')

doc = snakemd.new_doc("activity")

db_params_dict = ConfigManager.get_db_params()
garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]
altitude_units = unit_strings[fitfile.units.UnitTypes.altitude]
temp_units = unit_strings[fitfile.units.UnitTypes.tempurature]

activity = Activities.get(garmin_act_db, activity_id)

doc.add_header(activity.name)
if activity.description:
    doc.add_paragraph(activity.description)

doc.add_header("Summary", 3)
start_loc = Location(activity.start_lat , activity.start_long)
end_loc = Location(activity.stop_lat , activity.stop_long)
doc.add_table(
    [
        'Id', 'Sport', 'Type', f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", 'Avg HR', 'Max HR', 'Avg RR', 'Max RR', f'Gained ({altitude_units})', f'Lost ({altitude_units})',
        "Calories", f"Avg Temp (deg {temp_units})", 'Start Location', 'End Location'
    ],
    [
        [activity_id, activity.sport, activity.type, activity.distance, activity.elapsed_time, activity.moving_time, activity.avg_hr, activity.max_hr,
         activity.avg_rr if activity.avg_rr else '', activity.max_rr if activity.max_rr else '',
         round(activity.ascent), round(activity.descent), activity.calories, activity.avg_temperature, snakemd.InlineText(str(start_loc), start_loc.to_google_maps_url()),
         snakemd.InlineText(str(end_loc), end_loc.to_google_maps_url())]
    ]
)

doc.add_header("Laps", 3)
laps = ActivityLaps.get_activity(garmin_act_db, activity_id)
lap_stats = []
for lap in laps:
    start_loc = Location(lap.start_lat , lap.start_long)
    end_loc = Location(lap.stop_lat , lap.stop_long)
    lap_stats.append(
        [
            lap.lap, lap.distance, lap.elapsed_time, lap.moving_time, lap.avg_hr, lap.max_hr, lap.avg_rr if lap.avg_rr else '', lap.max_rr if lap.max_rr else '',
            round(lap.ascent), round(lap.descent), lap.calories, lap.avg_temperature,
            snakemd.InlineText(str(start_loc), start_loc.to_google_maps_url()), snakemd.InlineText(str(end_loc), end_loc.to_google_maps_url())
        ]
    )
doc.add_table(
    [
        'Lap', f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", 'Avg HR', 'Max HR', 'Avg RR', 'Max RR', f'Gained ({altitude_units})',
        f'Lost ({altitude_units})', "Calories", f"Avg Temp (deg {temp_units})", 'Start Location', 'End Location'
    ],
    lap_stats
)

doc.add_header("Time In Heart Rate Zones", 3)
doc.add_table(
    ['Zone', 'HR Threshold', "Time"],
    [
        ["1", activity.hrz_1_hr, activity.hrz_1_time],
        ["2", activity.hrz_2_hr, activity.hrz_2_time],
        ["3", activity.hrz_3_hr, activity.hrz_3_time],
        ["4", activity.hrz_4_hr, activity.hrz_4_time],
        ["5", activity.hrz_5_hr, activity.hrz_5_time],
    ])

doc.add_header("Training Effect", 3)
doc.add_table(
    ['Type', 'Rating'],
    [
        ["Aerobic", activity.training_effect],
        ["Anaerobic", activity.anaerobic_training_effect],
    ])

display(Markdown(str(doc)))